<a href="https://colab.research.google.com/github/shubhamksingh1/RecommendationEngine/blob/main/CollaborativeFiltering/Perspective/Keras/Test_Collaborative_User_Based_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import pickle
import pandas as pd
from keras.models import load_model 
import numpy as np

# Read Model Data and pickles

In [ ]:
model = load_model('/content/drive/MyDrive/Collaborative_Recommendation/Keras/Model/modelv1')

In [ ]:
asset_df = pd.read_pickle("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/recommendation_v4.pkl")
df = pd.read_pickle("/content/drive/MyDrive/Collaborative_Recommendation/Keras/ratingsv1keras.pkl")


In [ ]:
asset_df

,asset_id,asset_description,asset_title,custom_stop_desc,custom_stop_title,asset_description_cleaned,asset_title_cleaned,Domain
0,1,peter drucker the legendary management scholar...,the ascent of the smarter faster lighter enter...,"['3a', 'a', 'an', 'on', 'at', 'is', 's', 'do',...",['of'],peter drucker legendary management scholar con...,ascent smarter faster lighter enterprise,CX
1,2,the software development community has been us...,agile s day has truly arrived,"['a', 'u', 'we', 'k', 'i', 'on', 'at', 'is', '...",['s'],software development community using term agil...,agile day truly arrived,Agile
2,3,many exciting new digital technologies have ev...,riding the cognitive wave,"['a', 'we', 'e', 'an', 'on', 'at', 'is', 'do',...",[],many exciting new digital technology evolved r...,riding cognitive wave,AI
3,4,we have all seen some eye bulging numbers in r...,using iot data to understand how your products...,"['hp', 'a', 'we', 'an', 'on', 'at', 'is', 's',...",['to'],seen eye bulging number recent year internet t...,using iot data understand product perform,IOT
4,5,you probably need little evidence to believe t...,the view inside our take on the customer exper...,"['a', 'we', 'e', 'an', 'i', 'on', 'at', 'is', ...",['on'],probably need little evidence believe experien...,view inside take customer experience revolution,CX
...,...,...,...,...,...,...,...,...
77,80,this report is based on tcs larger study on di...,tcs cmo study how retailers are digitally pers...,"['3a', 'a', 'we', 'on', 'at', 'is', 's', 'do',...",['to'],report based tc larger study digital marketing...,tc cmo study retailer digitally personalizing ...,Industry
78,81,this report on marketers in the travel transpo...,tcs cmo study how travel transportation and ho...,"['a', 'we', 'e', 'on', 'is', 's', 'do', 'go', ...",['to'],report marketer travel transportation hospital...,tc cmo study travel transportation hospitality...,Industry
79,83,chief marketing officers have one of the riski...,tcs cmo study stage conversion personalizing c...,"['a', 'on', 'at', 'is', 's', 'do', '■', 'vs', ...",['to'],chief marketing officer one riskiest role suit...,tc cmo study stage conversion personalizing co...,Industry
80,84,the rise of the chief medical officer typicall...,employee wellness analysis creating data drive...,"['a', 'an', 'on', 'at', 'is', 's', 'up', 'by',...",[],rise chief medical officer typically business ...,employee wellness analysis creating data drive...,HR


In [ ]:
df

,uid,asset_id,Ratings,user,asset,rating
296,37,4,4,36,3,4.0
81,7,22,4,6,21,4.0
77,7,15,4,6,14,4.0
208,21,36,3,20,35,3.0
318,43,19,5,42,18,5.0
...,...,...,...,...,...,...
71,6,71,5,5,68,5.0
106,9,22,2,8,21,2.0
270,32,3,5,31,2,5.0
435,72,24,5,71,23,5.0


In [ ]:
asset2asset_encoded = pickle.load(open("/content/drive/MyDrive/Collaborative_Recommendation/Keras/asset2asset_encoded.pkl", "rb" )) 
user2user_encoded = pickle.load(open("/content/drive/MyDrive/Collaborative_Recommendation/Keras/user2user_encoded.pkl", "rb" ) )    
asset_encoded2asset= pickle.load(open("/content/drive/MyDrive/Collaborative_Recommendation/Keras/asset_encode2asset.pkl","rb"))

In [ ]:
print(asset2asset_encoded)
print(user2user_encoded)
print(asset_encoded2asset)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 59: 57, 60: 58, 61: 59, 62: 60, 63: 61, 65: 62, 66: 63, 67: 64, 68: 65, 69: 66, 70: 67, 71: 68, 72: 69, 73: 70, 74: 71, 75: 72, 76: 73, 77: 74, 78: 75, 79: 76, 80: 77, 81: 78, 83: 79, 84: 80, 85: 81}
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 

# Function Definition

In [ ]:
def get_recommendation(user_id,n):
            
    asset_read_by_user = df[df.uid == user_id]

    asset_not_read = asset_df[
        ~asset_df["asset_id"].isin(asset_read_by_user.asset_id.values)
    ]["asset_id"]

    asset_not_read = list(
        set(asset_not_read).intersection(set(asset2asset_encoded.keys()))
    )

    asset_not_read = [[asset2asset_encoded.get(x)] for x in asset_not_read]

    user_encoder = user2user_encoded.get(user_id)

    user_asset_array = np.hstack(
        ([[user_encoder]] * len(asset_not_read), asset_not_read)
    )


    ratings = model.predict(user_asset_array).flatten()

    top_ratings_indices = ratings.argsort()[-n:][::-1]

    recommended_asset_ids = [
        asset_encoded2asset.get(asset_not_read[x][0]) for x in top_ratings_indices
    ]

    print("Showing recommendations for user: {}".format(user_id))
    print("====" * 9)
    print("Assets with high ratings from user")
    print("----" * 8)
    top_assets_user = (
        asset_read_by_user.sort_values(by="rating", ascending=False)
        .head()
        .asset_id.values
    )
    asset_df_rows = asset_df[asset_df["asset_id"].isin(top_assets_user)]
    for row in asset_df_rows.itertuples():
        print(row.asset_id, ":", row.asset_title, ":",row.Domain)

    print("----" * 8)
    print("Top 5 article recommendations")
    print("----" * 8)

    for id in recommended_asset_ids:
        recommended_assets = asset_df[asset_df["asset_id"].isin([id])]
        for row in recommended_assets.itertuples():
            print(row.asset_id, ":", row.asset_title, ":",row.Domain)


# Get Recommendation

Users      Domain
*   1 - 12 :CX
*   13 - 24 : Agile
*   25 - 36 : AI
*   37 - 48 : IOT
*   49 - 60 : BigData
*   61 - 72 : Cloud
*   73 - 84 : HR
*   85 - 100 : Industry

In [ ]:
get_recommendation(1,5)

Showing recommendations for user: 1
Assets with high ratings from user
--------------------------------
17 : personalization boundaries tuning in to customers without turning them off : CX
18 : designing for the small silver screen : CX
20 : reimagining the contact center with digital technologies : CX
71 : how blockchain is poised to revolutionize digital customer experience : CX
76 : how quality engineering enables customer experience transformations : CX
--------------------------------
Top 5 article recommendations
--------------------------------
84 : employee wellness analysis creating data driven hr policies that help save lives : HR
83 : tcs cmo study stage conversion personalizing content to turn prospects into customers : Industry
79 : tcs cmo study stage awareness attracting the digitally distracted prospect : Industry
85 : why the emotional wellbeing of your employees should be a top priority during covid : HR
74 : proactive protection how companies can secure customer data

In [ ]:
get_recommendation(14, 5)

Showing recommendations for user: 14
Assets with high ratings from user
--------------------------------
31 : how to make location independent agile work : Agile
33 : enterprise agility pushing innovation to the edge of the organization : Agile
35 : how retail ceos can drive agile to grow their business : Agile
36 : fending off the fintechs how agile financial services firms are transforming their businesses : Agile
43 : why agile software development requires radical changes in budgeting and scoping : Agile
--------------------------------
Top 5 article recommendations
--------------------------------
18 : designing for the small silver screen : CX
11 : winning today s talent race how to identify develop and retain talent better than your rivals : HR
13 : how microsoft uses machine learning to handle workloads that humans cannot : AI
7 : case study why artificial intelligence is a big part of ap s future : AI
8 : banking and financial services analytics for competitive advantage : Big

In [ ]:
get_recommendation(28, 5)

Showing recommendations for user: 28
Assets with high ratings from user
--------------------------------
3 : riding the cognitive wave : AI
7 : case study why artificial intelligence is a big part of ap s future : AI
13 : how microsoft uses machine learning to handle workloads that humans cannot : AI
38 : where to turn ai and automation loose in your company : AI
42 : what happens when you turn your products into services : AI
--------------------------------
Top 5 article recommendations
--------------------------------
18 : designing for the small silver screen : CX
11 : winning today s talent race how to identify develop and retain talent better than your rivals : HR
8 : banking and financial services analytics for competitive advantage : BigData
1 : the ascent of the smarter faster lighter enterprise : CX
20 : reimagining the contact center with digital technologies : CX


In [ ]:
get_recommendation(47,5)

Showing recommendations for user: 47
Assets with high ratings from user
--------------------------------
19 : how iot technologies are revamping the post sale experience : IOT
41 : the magic that happens when manufacturers digitally attach themselves to their products : IOT
44 : raising your iot security game : IOT
50 : using iot technologies to completely rethink the customer experience interview with ric merrifield : IOT
--------------------------------
Top 5 article recommendations
--------------------------------
18 : designing for the small silver screen : CX
11 : winning today s talent race how to identify develop and retain talent better than your rivals : HR
13 : how microsoft uses machine learning to handle workloads that humans cannot : AI
7 : case study why artificial intelligence is a big part of ap s future : AI
8 : banking and financial services analytics for competitive advantage : BigData


In [ ]:
get_recommendation(55,5)

Showing recommendations for user: 55
Assets with high ratings from user
--------------------------------
8 : banking and financial services analytics for competitive advantage : BigData
47 : analytics a still emerging business frontier interview with prabhudev konana : BigData
61 : operating in an insight driven world mastering the new game of analytics : BigData
--------------------------------
Top 5 article recommendations
--------------------------------
71 : how blockchain is poised to revolutionize digital customer experience : CX
55 : digital data s greatest opportunities : BigData
41 : the magic that happens when manufacturers digitally attach themselves to their products : IOT
59 : taking a machine first approach to digital transformation : AI
67 : unleashing business value in a business world : Industry


In [ ]:
get_recommendation(63,5)

Showing recommendations for user: 63
Assets with high ratings from user
--------------------------------
27 : cloudera the mind of the new machine : Cloud
40 : now you can simulate nearly anything : Cloud
60 : immense digital power on demand shifting to the brave new world of cloud computing : Cloud
72 : for companies today the question is not whether to move to the cloud but how : Cloud
73 : how the cloud empowers digital transformation : Cloud
--------------------------------
Top 5 article recommendations
--------------------------------
55 : digital data s greatest opportunities : BigData
41 : the magic that happens when manufacturers digitally attach themselves to their products : IOT
59 : taking a machine first approach to digital transformation : AI
67 : unleashing business value in a business world : Industry
56 : how the digital renovators are racing ahead : Industry


In [ ]:
get_recommendation(77,5)

Showing recommendations for user: 77
Assets with high ratings from user
--------------------------------
11 : winning today s talent race how to identify develop and retain talent better than your rivals : HR
29 : students for life why executives must continually update their wisdom : HR
45 : executive traits for recognizing the bountiful opportunities ahead : HR
85 : why the emotional wellbeing of your employees should be a top priority during covid : HR
--------------------------------
Top 5 article recommendations
--------------------------------
55 : digital data s greatest opportunities : BigData
41 : the magic that happens when manufacturers digitally attach themselves to their products : IOT
59 : taking a machine first approach to digital transformation : AI
67 : unleashing business value in a business world : Industry
56 : how the digital renovators are racing ahead : Industry


In [ ]:
get_recommendation(80,5)

Showing recommendations for user: 80
Assets with high ratings from user
--------------------------------
11 : winning today s talent race how to identify develop and retain talent better than your rivals : HR
45 : executive traits for recognizing the bountiful opportunities ahead : HR
84 : employee wellness analysis creating data driven hr policies that help save lives : HR
--------------------------------
Top 5 article recommendations
--------------------------------
18 : designing for the small silver screen : CX
13 : how microsoft uses machine learning to handle workloads that humans cannot : AI
7 : case study why artificial intelligence is a big part of ap s future : AI
8 : banking and financial services analytics for competitive advantage : BigData
1 : the ascent of the smarter faster lighter enterprise : CX


In [ ]:
get_recommendation(95,5)

Showing recommendations for user: 95
Assets with high ratings from user
--------------------------------
28 : manufacturing how to lighten up business models : Industry
32 : the view inside technologies collide and industries transform : Industry
46 : changing culture in a time of accelerating market change interview with vijay govindarajan : Industry
51 : increasing speed in the digital era interview with jeanne ross : Industry
81 : tcs cmo study how travel transportation and hospitality companies are digitally personalizing content to create market awareness : Industry
--------------------------------
Top 5 article recommendations
--------------------------------
84 : employee wellness analysis creating data driven hr policies that help save lives : HR
83 : tcs cmo study stage conversion personalizing content to turn prospects into customers : Industry
85 : why the emotional wellbeing of your employees should be a top priority during covid : HR
76 : how quality engineering enables cu